In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from keras.datasets import imdb

Using TensorFlow backend.
C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [84]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000) #讀入最常出現的一萬字

In [85]:
from keras.preprocessing import sequence

In [86]:
x_train = sequence.pad_sequences(x_train, maxlen=100) #只取前100字判斷情緒
x_test = sequence.pad_sequences(x_test, maxlen=100)

In [87]:
x_train.shape

(25000, 100)

In [88]:
y_train.shape

(25000,)

**以CNN應用在情意分析上建構** 這裡選用 LSTM

- 先將 10000 維的文字壓到 128 維 (影評會依出現頻率大小，分別給予1-10000的標籤，故一個字有10000維，現在要embedded成128維)

- 做 2 次 convolution, 每次都接 max-pooling

- filter 大小都是 3, max-pooling 都用 2 為一小區塊

CNN 是每層的 filters 數目要越來越多。做完 convolution 之後要拉平、再送入一個標準的神經網路。這個神經網路設計是這樣:

- 1 個隱藏層, 使用 200 個神經元。

In [67]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.layers import LSTM, Conv1D, MaxPooling1D

In [90]:
model = Sequential()
model.add(Embedding(10000,128,input_length = 100)) #10000維(input_dim) ->128維(output_dim)，Embedding可將離散資料變連續

model.add(Conv1D(filters = 32, kernel_size = (3,) , activation = 'relu', padding='same'))
model.add(MaxPooling1D(pool_size=(2,)))

model.add(Conv1D(filters = 64, kernel_size = (3,), activation = 'relu',padding='same'))
model.add(MaxPooling1D(pool_size=(2,)))

#標準神經網路需輸入向量
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid')) #200個神經元
model.summary()

Model: "sequential_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_46 (Embedding)     (None, 100, 128)          1280000   
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 100, 32)           12320     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 50, 64)            6208      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 25, 64)            0         
_________________________________________________________________
flatten_36 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)               

In [91]:
model.compile(loss = 'binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

model.fit(x_train, y_train,batch_size=32,epochs=15)

Epoch 1/15
25000/25000 [==============================] - 27s 1ms/step - loss: 0.4067 - accuracy: 0.7999
Epoch 2/15
25000/25000 [==============================] - 27s 1ms/step - loss: 0.2158 - accuracy: 0.9160
Epoch 3/15
25000/25000 [==============================] - 28s 1ms/step - loss: 0.1097 - accuracy: 0.9596
Epoch 4/15
25000/25000 [==============================] - 23s 920us/step - loss: 0.0394 - accuracy: 0.9878
Epoch 5/15
25000/25000 [==============================] - 21s 830us/step - loss: 0.0197 - accuracy: 0.9933
Epoch 6/15
25000/25000 [==============================] - 21s 835us/step - loss: 0.0085 - accuracy: 0.9975
Epoch 7/15
25000/25000 [==============================] - 21s 830us/step - loss: 0.0097 - accuracy: 0.9968
Epoch 8/15
25000/25000 [==============================] - 21s 827us/step - loss: 0.0150 - accuracy: 0.9944
Epoch 9/15
25000/25000 [==============================] - 21s 838us/step - loss: 0.0099 - accuracy: 0.9964
Epoch 10/15
25000/25000 [==================

In [94]:
score= model.evaluate(x_test,y_test)

25000/25000 [==============================] - 3s 117us/step


In [96]:
print('loss:',score[0],
     '準確率:',score[1])

loss: 2.045762177672386 準確率: 0.8120399713516235
